# Mitigating State Leakage Across Reset Gates

Copyright 2022 Allen Mi, Shuwen Deng, and Jakub Szefer

This program is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with this program. If not, see <https://www.gnu.org/licenses/>.

## 1 - Imports and Definitions

Import libraries

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../')

In [ ]:
from itertools import combinations, product
import math

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import curve_fit, fsolve

from qiskit import QuantumCircuit, transpile, execute

import scripts.utils as u

Load IBM Q Credentials

In [ ]:
provider = u.load_provider('../credentials/provider.json')

Specify saved data and figures directories

In [ ]:
data_dir = '../experiments/inference/lagos_1q_32r_72c_1p'
figures_dir = '../figures/inference/lagos_1q_32r_72c_1p'

Specify constants
- Total number of qubits
- Width of two-column figures

In [ ]:
N_QUBITS = 7
FIG_SINGLE_WIDTH = 714
FIG_DOUBLE_WIDTH = 1500

Specify global variables
- Maximum number of reset operations
- Number of shots to be performed in experiments
- Number of $\theta$ samples in $[0, \pi]$
- Number of $\varphi$ samples in $[0, 2\pi)$
- Number of experiment passes

In [ ]:
MAX_N_RESETS = 32
N_SHOTS = 8192
N_THETA = 9
N_PHI = 8
N_PASSES = 1

Specify backend names

In [ ]:
be_name_list = ['ibm_lagos']

Specify backend proper names

In [ ]:
be_proper_name_dict = {
    'ibm_perth': 'Perth',
    'ibm_lagos': 'Lagos',
    'ibmq_jakarta': 'Jakarta'
}

## 2 - Experiments

### 2.1 - Defining Methods

In [ ]:
def make_config_list(n_theta, n_phi):
    theta_arr = np.linspace(0, np.pi, n_theta, endpoint=True)
    phi_arr = np.linspace(0, 2 * np.pi, n_phi, endpoint=False)

    return list(product(theta_arr, phi_arr))


def construct_circ(q, theta, phi, measure_first, reset_count):
    c = QuantumCircuit(7, 2)
    c.rx(theta, q)
    c.rz(phi, q)
    if measure_first:
        c.measure(q, 0)
    c.delay(4512 * 500, q)
    c.measure(q, 1)
    
    return c


def make_circuit_dict(config_arr, max_n_resets, n_passes):
    cd = {'meas': []}

    for q in range(1):
        cd['meas'].append([])
        for r in range(max_n_resets + 1):
            cd['meas'][q].append([
                construct_circ(q, theta, phi, True, r)
                for _, (theta, phi) in product(range(n_passes), config_arr)
            ])
    return cd


exec = lambda c, be_name, shots: execute(
    transpile(c, backend=provider.get_backend(be_name), optimization_level=0),
    backend=provider.get_backend(be_name), optimization_level=0,
    shots=shots
)

def make_all_jobs(circuit_dict, max_n_resets, n_shots):
    all_jd = {}

    for be_name in ['ibm_lagos']:
        jd = {'meas': []}
        for q in range(1):
            jd['meas'].append([])
            for r in range(1):
                jd['meas'][q].append(
                    exec(circuit_dict['meas'][q][r], be_name, n_shots)
                )
        all_jd[be_name] = jd
    
    return all_jd

In [ ]:
def get_counts(job):
    cnt = job.result().get_counts()
    v1 = [r.get('01', 0) + r.get('11', 0) for r in cnt]
    a1 = [r.get('10', 0) + r.get('11', 0) for r in cnt]
    
    return v1, a1

In [ ]:
def make_results(all_jobs, configs, n_shots, n_passes, is_idle=False, n_resets_override=None):
    pre = []
    
    for be_name, x in all_jobs.items():
        for victim_op, y in x.items():
            for i_q, z in enumerate(y):
                for i_r, job in enumerate(z):
                    if n_resets_override is not None:
                        i_r = n_resets_override[i_r]
                    v1, a1 = get_counts(job)
                    pre.extend([[
                        be_name, i_q,
                        victim_op, is_idle,
                        i_r,
                        theta, phi,
                        i_pass,
                        v1[i_c], v1[i_c] / n_shots,
                        a1[i_c], a1[i_c] / n_shots
                    ] for i_c, (i_pass, (theta, phi)) in enumerate(
                        product(range(n_passes), configs)
                    )])
    
    return pd.DataFrame(
        pre,
        columns=[
            'backend', 'qubit',
            'victim_op', 'is_idle',
            'n_resets',
            'theta', 'phi',
            'i_pass',
            'victim_count', 'victim_frequency',
            'attacker_count', 'attacker_frequency'
        ]
    )

### 2.2 - Thermalization (Idle) Results

In [ ]:
idle_results_list = []

# idles equivalent to 0 to 12 resets, each with 72 configurations
configs = make_config_list(N_THETA, N_PHI)
jobs = u.load(f'{data_dir}/idle_0-12@1.pickle')
jobs['ibm_lagos']['meas'][0] = jobs['ibm_lagos']['meas'][0][:13]
idle_results_list.append(
    make_results(jobs, configs, N_SHOTS, N_PASSES, is_idle=True, n_resets_override=range(13))
)

# idles equivalent to 16 to 32 resets, in intervals of 4 resets, each with 72 configurations
configs = make_config_list(N_THETA, N_PHI)
jobs = u.load(f'{data_dir}/idle_16-32@4.pickle')
idle_results_list.append(
    make_results(jobs, configs, N_SHOTS, N_PASSES, is_idle=True, n_resets_override=range(16, 33, 4))
)

# idles equivalent to 250, 1000, 2000, 4000, 8000 resets, each with 2 configurations (theta = 0 or pi)
configs = make_config_list(2, 1)
for n_resets_standalone in [250, 1000, 2000, 4000, 8000]:
    jobs = u.load(f'{data_dir}/idle_{n_resets_standalone}.pickle')
    idle_results_list.append(
        make_results(jobs, configs, N_SHOTS, N_PASSES, is_idle=True, n_resets_override=[n_resets_standalone])
    )

In [ ]:
idle_results = pd.concat(idle_results_list, ignore_index=True)
u.save(idle_results, f'{data_dir}/idle_results.pickle')

In [ ]:
idle_results = u.load(f'{data_dir}/idle_results.pickle')

### 2.3 - Repeated Resets Results

In [ ]:
jobs = u.load(f'{data_dir}/jobs.pickle')
jobs = {'ibm_lagos': {'meas': [jobs['ibm_lagos']['meas'][0]]}}

In [ ]:
configs = make_config_list(N_THETA, N_PHI)
reset_results = make_results(jobs, configs, N_SHOTS, N_PASSES)
u.save(reset_results, f'{data_dir}/reset_results.pickle')

In [ ]:
reset_results = u.load(f'{data_dir}/reset_results.pickle')

In [ ]:
raw_results = pd.concat([reset_results, idle_results.iloc[:-10]], ignore_index=True)

In [ ]:
raw_results = raw_results[
    (
        (raw_results.theta==0) |
        (raw_results.theta==np.pi)
    )
]
raw_results.reset_index(drop=True, inplace=True)

In [ ]:
mean_list = []
for r in range(0, len(raw_results), N_PHI):
    h = raw_results.iloc[[r]]
    s = raw_results.iloc[r: r + N_PHI]
    victim_count = s.victim_count.sum()
    victim_frequency = s.victim_frequency.mean()
    attacker_count = s.attacker_count.sum()
    attacker_frequency = s.attacker_frequency.mean()
    if h.is_idle.values[0]:
        typ = 'Thermalization'
    else:
        typ = 'Repeated Resets'
    mean_list.append([
        h.backend.values[0], typ,
        h.n_resets.values[0], h.theta.values[0],
        victim_count, victim_frequency,
        attacker_count, attacker_frequency
    ])
results = pd.DataFrame(
    mean_list,
    columns=[
        'backend', 'type', 'n_resets', 'theta', 'victim_count', 'victim_frequency', 'attacker_count', 'attacker_frequency'
    ]
)

In [ ]:
# add victim state label
results['victim_state'] = (results.theta == np.pi).astype(int).astype(str)

In [ ]:
# add std
results['attacker_std'] = np.sqrt(results.attacker_frequency * (1 - results.attacker_frequency) / N_SHOTS / 8)

In [ ]:
# add security parameter and K-L divergence
results['sec_param'] = 0
results['kl_divergence'] = 0
for i, r0 in results.iterrows():
    if i % 2 == 0:
        r1 = results.iloc[[i + 1]]
        p0 = r0.attacker_frequency
        p1 = r1.attacker_frequency
        sec_param = (p1 - p0) / (r1.attacker_std + r0.attacker_std)
        kl_divergence = N_SHOTS * 8 * (p0 * np.log(p0 / p1) + (1 - p0) * np.log((1 - p0) / (1 - p1)))
        results.iloc[i, -2] = sec_param
        results.iloc[i + 1, -2] = sec_param
        results.iloc[i, -1] = kl_divergence
        results.iloc[i + 1, -1] = kl_divergence
        

# absolute value
results['sec_param_abs'] = abs(results['sec_param'])

# direction
results['sec_param_is_forward'] = results['sec_param'] > 0

In [ ]:
freq_arr = np.zeros([32, 2])
kl_arr = np.zeros(32, dtype=int)
for r in range(32):
    s = results[results.type=='Repeated Resets'].iloc[2 * r: 2 * r + 2]
    freq_arr[r] = s['attacker_frequency'].values
    kl_arr[r] = s['kl_divergence'].values[0]

In [ ]:
best_kl = np.zeros_like(kl_arr)
for r in range(32):
    kl = kl_arr[:r + 1]
    best_kl[r] = kl.argmin()

### 2.4 - Choices for Reset Sequence

In [ ]:
choices = []
for r in range(32):
    fr = freq_arr[:r + 1]
    combs = list(combinations(enumerate(fr), 2))
    if not combs:
        choices.append({r: 1.0})
        continue
    best_mean = np.inf
    best_choice = {}
    for (i, pi), (j, pj) in combs:
        di = pi[1] - pi[0]
        dj = pj[1] - pj[0]
        if di == 0:
            choices.append({i: 1.0})
            break
        elif dj == 0:
            choices.append({j: 1.0})
            break
        elif np.sign(di) != np.sign(dj):
            fi = -dj / (di - dj)
            fj = di / (di - dj)
            assert np.isclose(fi * di, -fj * dj)
            assert np.isclose(fi + fj, 1)
            mean = fi * pi.mean() + fj * pj.mean()
            if mean < best_mean:
                best_choice = {i: fi, j: fj}
                best_mean = mean
    if not best_choice:
        best_choice = {best_kl[r]: 1.0}
            
    choices.append(best_choice)

In [ ]:
add_list = []
for r in range(32):
    f0 = 0
    f1 = 0
    for k, v in choices[r].items():
        f0 += v * freq_arr[k][0]
        f1 += v * freq_arr[k][1]
    if len(choices[r]) > 1:
        assert np.isclose(f0, f1)
        kl = 0
    else:
        kl = N_SHOTS * 8 * (f0 * np.log(f0 / f1) + (1 - f0) * np.log((1 - f0) / (1 - f1)))
    for f, vs in [(f0, '0'), (f1, '1')]:
        add_list.append(
            [
                'ibm_lagos', 'Secure Reset', r, 0.0,
                np.nan, np.nan, np.nan, f,
                vs, np.nan, np.nan,
                kl,
                np.nan, np.nan
            ]
        )

### 2.5 - Secure Reset Results

In [ ]:
secure_results = pd.DataFrame(
    add_list,
    columns=[
        'backend', 'type', 'n_resets', 'theta', 'victim_count', 'victim_frequency', 'attacker_count', 'attacker_frequency',
        'victim_state', 'attacker_std', 'sec_param', 'kl_divergence', 'sec_param_abs', 'sec_param_is_forward'
    ]
)

In [ ]:
results = pd.concat([results, secure_results], ignore_index=True)

## 2.6 - Plotting

Figure 10

In [ ]:
fig = px.scatter(
    results[
        (results.n_resets<=32) &
        (results.victim_state=='0')
    ],
    x='n_resets', y='kl_divergence', color='type', log_y=True,
    trendline='lowess', trendline_options=dict(frac=0.1),
    category_orders={'type': ['Thermalization', 'Repeated Resets', 'Secure Reset']},
    labels={
        'type': 'Reset Type',
        'kl_divergence': 'K-L Divergence',
        'n_resets': 'Maximum Time Budget, in Number of Resets'
    },
    title='K-L Divergence of Attacker Measurements Between 0/1-Output Victims',
)
fig.add_hline(
    y=0.1,
    line_dash='dash',
)
fig.update_layout(
    margin=dict(l=0, r=0, t=55, b=50, pad=0),
    height=400,
)
fig.show()

In [ ]:
fig.write_image(f'{figures_dir}/kl_divergence.pdf', width=FIG_SINGLE_WIDTH)

Figure 11

In [ ]:
fig = px.scatter(
    results[
        (results.n_resets<=32) &
        (results.victim_state=='1')
    ],
    x='n_resets', y='attacker_frequency', color='type', log_y=True,
    trendline='lowess', trendline_options=dict(frac=0.1),
    category_orders={'type': ['Thermalization', 'Repeated Resets', 'Secure Reset']},
    labels={
        'type': 'Reset Type',
        'attacker_frequency': 'Attacker 1-Output Frequency',
        'n_resets': 'Maximum Time Budget, in Number of Resets'
    },
    title='Attacker 1-Output Frequency Given 1-Output Victim',
)
fig.add_hline(
    y=0.015,
    line_dash='dash',
)
fig.update_layout(
    margin=dict(l=0, r=0, t=55, b=50, pad=0),
    height=400,
)
fig.show()

In [ ]:
fig.write_image(f'{figures_dir}/freq.pdf', width=FIG_SINGLE_WIDTH)

Figure 7

In [ ]:
fig = px.scatter(
    results[
        (results.n_resets >= 1) &
        (results.type=='Repeated Resets')
    ],
    x='n_resets', y='attacker_frequency', color='victim_state', error_y='attacker_std',
    log_x=False, log_y=True,
    trendline='lowess', trendline_options=dict(frac=0.1),
    labels={
        'victim_state': 'Victim Output',
        'attacker_frequency': 'Attacker 1-Output Frequency',
        'n_resets': 'Number of Repeated Resets'
    },
    title='Extended State Retention - Lagos, up to 32 Repeated Resets',
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_layout(legend=dict(
    yanchor="top",
    y=1,
    xanchor="right",
    x=1
))
fig.update_layout(
    margin=dict(l=0, r=0, t=55, b=50, pad=0),
    height=300,
)
fig.show()

In [ ]:
fig.write_image(f'{figures_dir}/sweep.pdf', width=FIG_DOUBLE_WIDTH)